In [53]:
# # read in the 3 txt files and combine into 1
# import glob
# import os

# # os.chdir('sample_data')
# read_files = glob.glob("sample_data/*.txt")

# with open("sample_data/combined.txt", "wb") as outfile:
#     for f in read_files:
#         with open(f, "rb") as infile:
#             outfile.write(infile.read())

In [54]:
# # import the dataset
# import pandas as pd
# import numpy as np
# import os

# # Directory path
# directory_path = 'C:\\Users\\azaan\\OneDrive\\Documents\\GitHub\\cs410_LLM_project\\sample_data'

# # Initialize an empty DataFrame
# df = pd.DataFrame(columns=['Filename', 'Content'])

# # Iterate over each file in the directory
# for filename in os.listdir(directory_path):
#     if filename.endswith('.txt'):
#         file_path = os.path.join(directory_path, filename)
        
#         # Read the content of the text file
#         with open(file_path, 'r') as file:
#             content = file.read()
        
#         # Append a row to the DataFrame
#         df = df.append({'Filename': filename, 'Content': content}, ignore_index=True)

# # Display the resulting DataFrame
# print(df)

  Filename                                            Content
0  7_1.txt  0:00\n[SOUND] In this lecture we give an overv...
1  7_2.txt  0:00\n[SOUND] So, looking at the text mining p...
2  7_3.txt  0:00\n[SOUND]\nPlay video starting at ::9 and ...


C:\Users\azaan\AppData\Local\Temp\ipykernel_32988\3844069920.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Filename': filename, 'Content': content}, ignore_index=True)
C:\Users\azaan\AppData\Local\Temp\ipykernel_32988\3844069920.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Filename': filename, 'Content': content}, ignore_index=True)
C:\Users\azaan\AppData\Local\Temp\ipykernel_32988\3844069920.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Filename': filename, 'Content': content}, ignore_index=True)


In [3]:
# import the dataset
import pandas as pd
import numpy as np
import os

# Directory path
directory_path = 'C:\\Users\\azaan\\OneDrive\\Documents\\GitHub\\cs410_LLM_project\\data\\all_lectures.csv'

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['Week Number', 'Lesson Number', 'Lesson Title', 'Transcript'])

# Read in csv to dataframe
df = pd.read_csv(directory_path)

# Display the resulting DataFrame
df.head()

,Week Number,Lesson Number,Lesson Title,Transcript
0,1,1,Natural Language Content Analysis,This lecture is about Natural Language of Cont...
1,1,2,Text Access,"In this lecture,\nwe're going to talk about th..."
2,1,3,Text Retrieval Problem,This lecture is about\nthe text retrieval prob...
3,1,4,Overview of Text Retrieval Methods,This lecture is a overview of\ntext retrieval ...
4,1,5,Vector Space Model - Basic Idea,This lecture is about the\nvector space retrie...


In [6]:
# clean up words in dataset -- this includes removing stopwords
import regex as re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, words, brown

nltk.download("stopwords")
nltk.download("words")
nltk.download("brown")

lemmer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# initialize dictionary
global_dictionary  = set(words.words()) | set(brown.words())
global_dictionary = {word.lower() for word in global_dictionary}
remove_words = list(stop_words) # might need to use word_tokenize
remove_words.extend(['Play', 'video', 'starting', 'at', '::', 'follow', 'transcript', 'natural', 'language', 'lecture', 'processing']) # add the common words that's include d in transcript

# Now start actually cleaning the text
def clean_text(text):
    text = text.lower() # lowercase
    text = text.replace('\n', ' ') # remove newline indicator
    text = re.sub(r'[^a-zA-Z\s]', '', text) # case
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'http\S+|www\S+', '', text) # website
    text = re.sub(r'(\b\w+\b)(?: \1)+', r'\1', text) # remove duplicate next word after space
    text = re.sub(r'\b(?![aI]\b)\w\b', '', text)

    return text

# Remove stopwords and only keep words in dictionary
def remove_terms(text):
    text = clean_text(text)
    words = text.split()
    filtered_words = [word for word in words if word not in remove_words] # remove stopwords
    filtered_words = [word for word in filtered_words if word in global_dictionary] # remove if not in global dictionary
    return " ".join(filtered_words)

df['Transcript_Cleaned'] = df['Transcript'].apply(remove_terms)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [8]:
df['Transcript_Cleaned'][1]

'going talk text access previous talked content analysis explained state techniques still good enough process lot unrestricted text data robust manner result bag words remains popular applications like search engine going talk help users get access text data also important step convert raw big text data small random data actually needed specific application main question well address text information system help users get access relevant text data going cover two complimentary push versus pull going talk two ways implement pull mode querying versus browsing first push versus pull two different ways connect users right information right time difference takes initiative party takes initiative pull mode users take initiative start information access process case user typically would use search engine fulfill goal example user may type query browse results find relevant information usually appropriate satisfying users ad hoc information need ad hoc information need temporary information ne

In [10]:
# Create bigrams and trigrams from data

# Function to filter bigrams or trigrams
def ngram_filter(ngram):
    tags = nltk.pos_tag(ngram)
    if not all(tag[1] in ['JJ', 'NN'] for tag in tags):
        return False
    if any(word in stop_words for word in ngram):
        return False
    if 'n' in ngram or 't' in ngram:
        return False
    if 'PRON' in ngram:
        return False
    return True

# Function to find top ngrams
def find_top_ngrams(texts, ngram_measures, min_freq=50, min_pmi=5, top_k=100):
    finder = nltk.collocations.BigramCollocationFinder.from_documents(texts)
    finder.apply_freq_filter(min_freq)
    ngram_scores = finder.score_ngrams(ngram_measures.pmi)
    filtered_ngrams = [ngram for ngram, pmi in ngram_scores if ngram_filter(ngram) and pmi > min_pmi]
    return [' '.join(ngram) for ngram in filtered_ngrams][:top_k]

bigram_measures = nltk.collocations.BigramAssocMeasures()
bigrams = find_top_ngrams([text.split() for text in df['Transcript_Cleaned']], bigram_measures)
trigram_measures = nltk.collocations.TrigramAssocMeasures()
trigrams = find_top_ngrams([text.split() for text in df['Transcript_Cleaned']], trigram_measures)

# Function to replace ngrams in text
def replace_ngrams(text):
    for gram in trigrams:
        text = text.replace(gram, '_'.join(gram.split()))
    for gram in bigrams:
        text = text.replace(gram, '_'.join(gram.split()))
    return text

# Apply ngram replacements to the text
df['Grams'] = df['Transcript_Cleaned'].map(replace_ngrams)


In [11]:
# Tokenize reviews + remove stop words + filter only nouns
def tokenize_and_filter(text):
    words = nltk.word_tokenize(text)
    words = [word.lower() for word in words if word.lower() not in stop_words and len(word) > 2]
    pos_comment = nltk.pos_tag(words)
    filtered = [word[0] for word in pos_comment if word[1] in ['NN']]
    return filtered

df['Grams'] = df['Grams'].map(tokenize_and_filter)

In [12]:
# some pytorch  training
import torch

randint = torch.randint(0, 10000, (6,1))
tensor = torch.tensor([[5, 8, 7],  [4, 8, 6], [9, 8, 4]])
zeros = torch.zeros([6,6])
ones = torch.ones([6,6])
arange = torch.arange(0, 10000)
linspace = torch.linspace(0, 10000, steps=100)
logspace = torch.logspace(-5000, 5000, steps=1000)
eye = torch.eye(6)
empty_like = torch.empty_like(eye)

randint
tensor
zeros
ones
arange
linspace
logspace
eye
empty_like

tensor([[1.0102e-38, 5.9694e-39, 1.0286e-38, 8.9081e-39, 8.9082e-39, 6.9796e-39],
        [9.0919e-39, 9.9184e-39, 7.3470e-39, 1.0194e-38, 1.0469e-38, 1.0010e-38],
        [8.4490e-39, 1.1112e-38, 9.5511e-39, 1.0102e-38, 7.3470e-39, 1.0653e-38],
        [1.0194e-38, 4.6838e-39, 4.4082e-39, 9.9184e-39, 9.0000e-39, 1.0561e-38],
        [1.0653e-38, 4.1327e-39, 8.9082e-39, 9.8265e-39, 9.4592e-39, 1.0561e-38],
        [1.0653e-38, 1.0469e-38, 9.5510e-39, 8.7245e-39, 8.9082e-39, 9.8266e-39]])

In [13]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)
import torch.nn as nn
import torch.nn.functional as F

# more pytorch fucntions
probabilities = torch.tensor([0.4, 0.6, 0.1])
samples = torch.multinomial(probabilities, num_samples=7, replacement=True)
ones = torch.ones([6,6])
tril = torch.tril(ones)
triu = torch.triu(ones)
zeros = torch.zeros(6, 6)
masked_fill = zeros.masked_fill(tril == 1, float(8))
exp = torch.exp(masked_fill)
transposed = tril.transpose(0, 1)
stack = torch.stack([tril, triu, zeros])
sample = torch.tensor([10., 10., 10.])
linear = nn.Linear(3, 3, bias=False)
sample_linear = linear(sample)
sample_softmax = F.softmax(sample)

samples
ones
tril
triu
zeros
masked_fill
exp
transposed
stack
sample
linear
sample_linear
sample_softmax

C:\Users\azaan\AppData\Local\Temp\ipykernel_45480\388226070.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sample_softmax = F.softmax(sample)


tensor([0.3333, 0.3333, 0.3333])

In [14]:
# now I will make embeddings for my words, let's see if it works
from torch.nn.utils.rnn import pad_sequence

results = set()
df['Grams'].apply(results.update)
# print(len(results))

# Create a vocabulary dictionary
word_to_index = {word: idx for idx, word in enumerate(results)}

# Convert words to indices in your DataFrame
# AKA Encode these
df['Grams_indices'] = df['Grams'].apply(lambda x: [word_to_index[word] for word in x])

# Create a reverse dictionary
index_to_word = {idx: word for word, idx in word_to_index.items()}

# Function to convert indices back to words
def indices_to_words(indices):
    return [index_to_word[idx] for idx in indices]

# # Apply the function to the 'Grams_indices' column
# Aka Decode these grams
# df['Decoded_Grams'] = df['Grams_indices'].apply(indices_to_words)

# Pad sequences to a specified length (e.g., maxlen)
maxlen = 50  # You can adjust this based on your data
padded_indices = pad_sequence([torch.LongTensor(seq) for seq in df['Grams_indices']], batch_first=True, padding_value=0)

In [15]:
# matrix multiplication examples with pytorch
a = torch.tensor([[3, 4], [5, 8], [8, 3]])
b = torch.tensor([[8, 26, 5], [34, 7, 17]])

# form matmul, can do either below
a @ b
matmul = torch.matmul(a, b)

matmul

tensor([[160, 106,  83],
        [312, 186, 161],
        [166, 229,  91]])

In [16]:
# make a batch
block_size = 8
batch_size = 4

# Flatten the padded indices used to identify each word
data = flattened_indices = padded_indices.view(-1)
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
# print(len(data))

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

x, y = get_batch('train')
print(x)
print(y)


tensor([[   0,    0,    0,    0,    0,    0,    0,    0],
        [   0,  812,  691, 1457,  528, 1342,  528,  812],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [1005, 1528,  513,  592, 1005,  191, 1437,  872]])
tensor([[   0,    0,    0,    0,    0,    0,    0,    0],
        [ 812,  691, 1457,  528, 1342,  528,  812,  691],
        [   0,    0,    0,    0,    0,    0,    0,    0],
        [1528,  513,  592, 1005,  191, 1437,  872,  191]])


In [17]:
# Now to make a simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probabilities = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probabilities, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

# embedding_dim = 11
# embedding = nn.Embedding(vocab_size, embedding_dim)

# embedded_indices = torch.LongTensor(df['Grams_indices'].tolist())
# embedded_grams = embedding(padded_indices)

In [18]:
vocab_size = len(results)


model = BigramLanguageModel(vocab_size)
context = torch.zeros((1, 1), dtype=torch.long)
generated_terms = indices_to_words(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_terms)

['examine', 'extent', 'unique', 'notion', 'spending', 'personalization', 'september', 'second', 'deviate', 'respond', 'era', 'randomness', 'oil', 'frame', 'library', 'know', 'param', 'hear', 'quit', 'paradigm', 'motion', 'pattern', 'background_model', 'somehow', 'retrieve', 'house', 'eigenvalue', 'week', 'symbolist', 'distribution', 'setting', 'resolve_problem', 'speed', 'table', 'simplicity', 'article', 'duplicate', 'recalibration', 'balance', 'review', 'diplomacy', 'non', 'signal', 'worth', 'technology', 'quit', 'war', 'connection', 'record', 'illustrate', 'discovering', 'efficient', 'suggestion', 'sociologist', 'robot', 'pseudo', 'advertising', 'schema', 'hit', 'ser', 'statement', 'think', 'invoke', 'airport', 'technology', 'threshold', 'animal', 'comparing', 'accurate', 'note', 'show', 'consumer', 'surprising', 'conclude', 'argument', 'index', 'proper', 'retrieval', 'inventory', 'recreation', 'guide', 'frame', 'module', 'backbone', 'symmetrical', 'dislike', 'angle', 'saturday', 'do

In [19]:
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

# Estimating losses function
@torch.no_grad()

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
            

In [20]:
# Creating an Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter}, train loss: {losses['train']}, val loss: {losses['val']}")
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


step 0, train loss: 7.715757369995117, val loss: 7.75339412689209
step 250, train loss: 7.627959728240967, val loss: 7.680568218231201
step 500, train loss: 7.561256408691406, val loss: 7.5812764167785645
step 750, train loss: 7.469468593597412, val loss: 7.532351970672607
step 1000, train loss: 7.353198528289795, val loss: 7.462738990783691
step 1250, train loss: 7.239411354064941, val loss: 7.38288688659668
step 1500, train loss: 7.17946195602417, val loss: 7.340051651000977
step 1750, train loss: 7.086147785186768, val loss: 7.255041122436523
step 2000, train loss: 6.9901580810546875, val loss: 7.194165229797363
step 2250, train loss: 6.93467378616333, val loss: 7.119497299194336
step 2500, train loss: 6.812793254852295, val loss: 7.031903266906738
step 2750, train loss: 6.6689863204956055, val loss: 6.9859395027160645
step 3000, train loss: 6.622453689575195, val loss: 6.903146743774414
step 3250, train loss: 6.540916442871094, val loss: 6.815154075622559
step 3500, train loss: 6.4

In [21]:
context = torch.zeros((1, 1), dtype=torch.long)
generated_terms = indices_to_words(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_terms)

['examine', 'discovering', 'indri', 'element', 'wont', 'closeness', 'delta', 'none', 'max', 'vector_space_model', 'write', 'criterion', 'accumulate', 'methodology', 'incorporate', 'robustness', 'copy', 'meaning', 'libation', 'get', 'mechanism', 'color', 'incorrect', 'data', 'complex', 'precise', 'tag', 'hit', 'order', 'attraction', 'compensate', 'width', 'era', 'filter', 'infinity', 'relevant', 'want', 'lag', 'proto', 'climb', 'stare', 'regime', 'pierre', 'regeneration', 'couple', 'table', 'training', 'format', 'commend', 'tell', 'reason', 'partial', 'attribute', 'disadvantage', 'independence', 'war', 'taxi', 'collector', 'independence', 'total', 'confidence', 'trade', 'definition', 'word', 'relation', 'didnt', 'incentive', 'recalculation', 'thing', 'state', 'accurate', 'usability', 'domain', 'climb', 'output', 'scientist', 'energy', 'classifier', 'complexity', 'word', 'detection', 'math', 'predictor', 'holder', 'margin', 'randomly', 'kind', 'chicago', 'notation', 'optimization', 'non'

In [22]:
input = torch.randn((4, 8, 10))
B, T, C = input.shape
output = input.view(B*T, C)
# print(output)
print(output[:][-1][:])

tensor([-1.6414,  1.5462, -0.1138, -0.1755,  0.1906,  0.4271,  0.3570,  2.2193,
         0.1779, -1.0558])


In [23]:
x = torch.tensor([-0.05], dtype=torch.float32)
y = nn.ReLU(x)
print(y)

x = torch.tensor([-0.05], dtype=torch.float32)
y = torch.sigmoid(x)
print(y)

x = torch.tensor([-0.05], dtype=torch.float32)
y = torch.tanh(x)
print(y)

ReLU(inplace=True)
tensor([0.4875])
tensor([-0.0500])
